In [32]:
from sklearn import model_selection , linear_model, metrics, pipeline,tree, preprocessing,neighbors,ensemble,svm,naive_bayes
from sklearn.model_selection import GridSearchCV,cross_val_score,train_test_split,RandomizedSearchCV,cross_val_predict
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler

#from sklearn import model_selection import grid_search
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from itertools import *
#from sklearn.linear_model import LogisticRegression
#from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [33]:
original_train_data = pd.read_csv('./data/train.csv')
original_test_data = pd.read_csv('./data/test.csv')

In [34]:
def prepare_data(data,mode): 
    
    #Обработка пропусков
    #Заполняем медианой
    #raw_data.Age = raw_data.Age.fillna(raw_data.Age.median())
    #Заполняем медианой c группировкой
    raw_data=data.copy()
    grp = raw_data.groupby(['Sex', 'Pclass'])  
    raw_data.Age = grp.Age.apply(lambda x: x.fillna(x.median()))
    #Заполняем модой
    #raw_data.Embarked = raw_data.Embarked.fillna(raw_data.Embarked.value_counts().idxmax())
    raw_data.Embarked.fillna(raw_data.Embarked.mode()[0], inplace = True)
    raw_data.Fare.fillna(raw_data.Fare.mode()[0],inplace = True)
    #Заполняем 'N'
    raw_data.Cabin = raw_data.Cabin.fillna('NA')

    
    if mode=='new_features':
        print('--new_features mode--')
        #raw_data.Embarked.value_counts()
        #добавим новые признаки
        raw_data['Family'] = raw_data.Parch + raw_data.SibSp+1
        raw_data['Family_Size'] = pd.cut(raw_data.Family, [0,1.5,4.5,15],labels=['Single','SmallFamily','LargeFamily'])
        #raw_data['Is_Alone'] = (raw_data.Family == 0).astype(int)
        raw_data['Salutation'] = raw_data.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip()) 
        raw_data['Age_Range'] = pd.cut(raw_data.Age, [0, 10, 20, 30, 40, 50, 60,70,80])
        raw_data['Fare_Category'] = pd.cut(raw_data.Fare, bins=[0,7.90,14.45,31.28,120.0,513.0],include_lowest = True, labels=['Low','Mid','High_Mid','High','Upper_High'])
        raw_data['isCabin']=raw_data.Cabin.apply(lambda x: 1 if x != 'NA' else 0)
        np.seterr(divide = 'ignore')
        raw_data['Log_Fare']=np.log10(raw_data.Fare).replace(-np.inf, 0)
        #raw_train_data.iloc[np.where(raw_train_data.Log_Fare.values<np.finfo(np.float64).min)]
        #raw_data['Fare_Category'] = pd.cut(raw_data['Fare'], bins=[0,7.90,14.45,31.28,120.0], labels=['Low','Mid','High_Mid','High'])
        #raw_data['Fare_Category'] = pd.cut(raw_data['Fare'], [0,7.90,14.45,31.28,120.0,513.0],include_lowest = True)
        #raw_data.Salutation.nunique()
        #####################New features#####################
        name_dict = {"Capt":       "officer",
                     "Col":        "officer",
                     "Major":      "officer",
                     "Dr":         "officer",
                     "Rev":        "officer",
                     "Jonkheer":   "snob",
                     "Don":        "snob",
                     "Sir" :       "snob",
                     "the Countess":"snob",
                     "Dona":       "snob",
                     "Lady" :      "snob",
                     "Mme":        "married",
                     "Ms":         "married",
                     "Mrs" :       "married",
                     "Miss" :      "single",
                     "Mlle":       "single",
                     "Mr" :        "man",
                     "Master" :    "boy"
                    }
        raw_data['Salutation_type'] = raw_data['Salutation'].map(name_dict)
        
    else:
        print('--Original mode--')
        pass
        
    return raw_data


In [35]:
def create_estimator(numeric_data_indices,categorical_data_indices,classifier):
    estimator = pipeline.Pipeline(steps = [       
        (
            'feature_processing', pipeline.FeatureUnion(transformer_list = [        
                #binary
                #('binary_variables_processing', preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])), 
                        
                #numeric
                ('numeric_variables_processing', pipeline.Pipeline(steps = [
                    ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, numeric_data_indices],validate=False)),
                    ('scaling', preprocessing.StandardScaler())            
                            ])),
        
                #categorical
                ('categorical_variables_processing', pipeline.Pipeline(steps = [
                    ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, categorical_data_indices],validate=False)),
                    ('hot_encoding', preprocessing.OneHotEncoder(handle_unknown = 'ignore',sparse=False))            
                            ])),
            ])
        ),
        ('model_fitting', classifier)
        ]
    )
    return estimator

## Train data

In [36]:
#Подготовка с отбором признаков

#exclude_col = ['Name', 'Embarked', 'Cabin', 'isCabin','Salutation', 'Sex', 'Family']
#['Name', 'Embarked', 'Cabin', 'isCabin'] ['Salutation', 'Sex', 'Family']
#prepare_data(raw_train_data,[])



train_data=prepare_data(original_train_data,'new_features')

train_labels = train_data['Survived']
train_data.drop(['PassengerId', 'Survived'], axis = 1,inplace=True)
all_column=train_data.columns
#Описание колонок и индексов для FeatureUnion
string_data_columns = ['Name','Ticket','Cabin']
categorical_data_columns = ['Pclass','Sex','Embarked','Salutation','Age_Range', 'Fare_Category','Family_Size','Salutation_type']
numeric_data_columns = ['Age', 'SibSp', 'Parch', 'Fare','Family','isCabin','Log_Fare']
categorical_data_indices = np.array([(column in categorical_data_columns) for column in train_data.columns], dtype = bool)
numeric_data_indices = np.array([(column in numeric_data_columns) for column in train_data.columns], dtype = bool)


--new_features mode--


### Отбор признаков

In [37]:
#classifier_dict = {'RiC':linear_model.RidgeClassifier(),'SVC':svm.SVC(gamma='auto') ,'RF':ensemble.RandomForestClassifier(n_estimators=10),'GB':ensemble.GradientBoostingClassifier(),'KNC':neighbors.KNeighborsClassifier(),'DeTree':tree.DecisionTreeClassifier()}
classifier_dict = {'SVC':svm.SVC(gamma='auto')}

In [38]:
#Сочетание без повторений
combinations_features_1=[[x] for x in all_column]
print('len combinations_features_1:',len(combinations_features_1))
combinations_features_2=[list(set().union(*x)) for x in combinations(combinations_features_1, 2)]
print('len combinations_features_2:',len(combinations_features_2))
combinations_features_3=[list(set().union(*x)) for x in combinations(all_column, 3)]
print('len combinations_features_3:',len(combinations_features_3))
combinations_features_all=[[]]+combinations_features_1+combinations_features_2+combinations_features_3
print('len combinations_features_all:',len(combinations_features_all))

len combinations_features_1: 18
len combinations_features_2: 153
len combinations_features_3: 816
len combinations_features_all: 988


In [39]:
%%time
# Перебор изъятия признаков


exclude_col_list=[]
clf_res_list=[]
#cv_res_list=[]
mean_res_list=[]
max_res_list=[]
min_res_list=[]
std_res_list=[]
#test_acc=[]
#for exclude_col in all_column:
for exclude_col in combinations_features_all:
    #exclude_col=list(set().union(*exclude_col_tup))
    print(exclude_col)
    
    current_train_data=train_data.copy()
    current_train_data.drop(exclude_col, axis = 1,inplace=True)
    current_categorical_data_indices = np.array([(column in categorical_data_columns) for column in current_train_data.columns], dtype = bool)
    current_numeric_data_indices = np.array([(column in numeric_data_columns) for column in current_train_data.columns], dtype = bool)

    for k, v in classifier_dict.items():
        #print("Code : {0}, Value : {1}".format(k, v))
        #print('***Classifier*****',k)
        clf=create_estimator(current_numeric_data_indices,current_categorical_data_indices,v)
        clf_scoring = cross_val_score(clf, current_train_data.values, train_labels, scoring = 'accuracy', cv = 3)
            #print ('mean:{}, max:{}, min:{}, std:{}'.format(clf_scoring.mean(), clf_scoring.max(), clf_scoring.min(), clf_scoring.std()))
        exclude_col_list.append(exclude_col)   
        clf_res_list.append(k)
        #cv_res_list.append(name)
        mean_res_list.append(clf_scoring.mean())
        max_res_list.append(clf_scoring.max())
        min_res_list.append(clf_scoring.min())
        std_res_list.append(clf_scoring.std())
        #clf.fit(X_train, y_train)
        #y_pred=clf.predict(X_test)
        #test_acc.append(accuracy_score(y_test, y_pred))
    

[]
['Pclass']
['Name']
['Sex']
['Age']
['SibSp']
['Parch']
['Ticket']
['Fare']
['Cabin']
['Embarked']
['Family']
['Family_Size']
['Salutation']
['Age_Range']
['Fare_Category']
['isCabin']
['Log_Fare']
['Salutation_type']
['Name', 'Pclass']
['Sex', 'Pclass']
['Pclass', 'Age']
['SibSp', 'Pclass']
['Parch', 'Pclass']
['Ticket', 'Pclass']
['Fare', 'Pclass']
['Cabin', 'Pclass']
['Embarked', 'Pclass']
['Family', 'Pclass']
['Pclass', 'Family_Size']
['Salutation', 'Pclass']
['Age_Range', 'Pclass']
['Fare_Category', 'Pclass']
['isCabin', 'Pclass']
['Log_Fare', 'Pclass']
['Salutation_type', 'Pclass']
['Sex', 'Name']
['Name', 'Age']
['SibSp', 'Name']
['Parch', 'Name']
['Ticket', 'Name']
['Fare', 'Name']
['Name', 'Cabin']
['Embarked', 'Name']
['Family', 'Name']
['Name', 'Family_Size']
['Salutation', 'Name']
['Age_Range', 'Name']
['Name', 'Fare_Category']
['isCabin', 'Name']
['Name', 'Log_Fare']
['Salutation_type', 'Name']
['Sex', 'Age']
['Sex', 'SibSp']
['Sex', 'Parch']
['Sex', 'Ticket']
['Sex', '

KeyError: "['m' 's' 'S' 'c' 'P' 'x' 'N' 'a' 'l' 'e'] not found in axis"

In [11]:
select_f_df=pd.DataFrame({'ex_col':exclude_col_list,'clf':clf_res_list,'mean':mean_res_list,'max':max_res_list,'min':min_res_list,'std':std_res_list})
select_f_df.sort_values(by=['mean'],ascending=False)#.head(20)

,ex_col,clf,mean,max,min,std
1417,"[Embarked, isCabin]",GB,0.840629,0.875421,0.804714,0.028877
1457,"[Family, Salutation]",GB,0.837262,0.855219,0.811448,0.018713
1257,"[Fare, Log_Fare]",GB,0.837262,0.848485,0.821549,0.011446
1057,"[Parch, Salutation_type]",GB,0.836139,0.858586,0.804714,0.022891
101,[Embarked],SVC,0.835017,0.841751,0.821549,0.009523
1,[],SVC,0.835017,0.841751,0.821549,0.009523
1421,"[Embarked, Log_Fare]",SVC,0.835017,0.841751,0.821549,0.009523
1511,"[Salutation, Family_Size]",SVC,0.835017,0.841751,0.824916,0.007274
1341,"[Log_Fare, Cabin]",SVC,0.835017,0.841751,0.821549,0.009523
431,"[Embarked, Name]",SVC,0.835017,0.841751,0.821549,0.009523


In [18]:
#select_f_df.to_csv('select_f_df_origin')
select_f_df.nlargest(10, 'mean')

,ex_col,clf,mean,max,min,std
1417,"[Embarked, isCabin]",GB,0.840629,0.875421,0.804714,0.028877
1257,"[Fare, Log_Fare]",GB,0.837262,0.848485,0.821549,0.011446
1457,"[Family, Salutation]",GB,0.837262,0.855219,0.811448,0.018713
1057,"[Parch, Salutation_type]",GB,0.836139,0.858586,0.804714,0.022891
1,[],SVC,0.835017,0.841751,0.821549,0.009523
21,[Name],SVC,0.835017,0.841751,0.821549,0.009523
71,[Ticket],SVC,0.835017,0.841751,0.821549,0.009523
91,[Cabin],SVC,0.835017,0.841751,0.821549,0.009523
101,[Embarked],SVC,0.835017,0.841751,0.821549,0.009523
151,[Fare_Category],SVC,0.835017,0.841751,0.821549,0.009523


In [19]:
select_f_df_max_10_mean=select_f_df[select_f_df.clf=='GB'].nlargest(10, 'mean').copy()
select_f_df_max_10_mean=select_f_df_max_10_mean.append(select_f_df[select_f_df.clf=='KNC'].nlargest(10, 'mean'))
select_f_df_max_10_mean=select_f_df_max_10_mean.append(select_f_df[select_f_df.clf=='DeTree'].nlargest(10, 'mean'))
select_f_df_max_10_mean=select_f_df_max_10_mean.append(select_f_df[select_f_df.clf=='DeTree'].nlargest(10, 'mean'))
select_f_df_max_10_mean=select_f_df_max_10_mean.append(select_f_df[select_f_df.clf=='RF'].nlargest(10, 'mean'))
select_f_df_max_10_mean=select_f_df_max_10_mean.append(select_f_df[select_f_df.clf=='SVC'].nlargest(10, 'mean'))
select_f_df_max_10_mean=select_f_df_max_10_mean.append(select_f_df[select_f_df.clf=='RiC'].nlargest(10, 'mean'))

In [20]:
select_f_df[select_f_df.clf=='SVC'].nlargest(10, 'mean')

,ex_col,clf,mean,max,min,std
1,[],SVC,0.835017,0.841751,0.821549,0.009523
21,[Name],SVC,0.835017,0.841751,0.821549,0.009523
71,[Ticket],SVC,0.835017,0.841751,0.821549,0.009523
91,[Cabin],SVC,0.835017,0.841751,0.821549,0.009523
101,[Embarked],SVC,0.835017,0.841751,0.821549,0.009523
151,[Fare_Category],SVC,0.835017,0.841751,0.821549,0.009523
161,[isCabin],SVC,0.835017,0.841751,0.821549,0.009523
171,[Log_Fare],SVC,0.835017,0.841751,0.821549,0.009523
401,"[Ticket, Name]",SVC,0.835017,0.841751,0.821549,0.009523
421,"[Name, Cabin]",SVC,0.835017,0.841751,0.821549,0.009523


In [21]:
select_f_df_max_10_mean.groupby(['clf']).max()

,ex_col,mean,max,min,std
clf,,,,,
DeTree,"[isCabin, Age]",0.796857,0.835017,0.774411,0.033332
GB,"[isCabin, Name]",0.840629,0.875421,0.821549,0.030407
KNC,"[Embarked, isCabin]",0.831650,0.868687,0.804714,0.038976
RF,"[SibSp, Name]",0.823793,0.845118,0.818182,0.023489
RiC,"[Embarked, Ticket]",0.829405,0.838384,0.818182,0.014285
SVC,[isCabin],0.835017,0.841751,0.821549,0.009523


In [22]:
select_f_df_max_10_mean.sort_values(by=['mean'],ascending=False).head(20)

,ex_col,clf,mean,max,min,std
1417,"[Embarked, isCabin]",GB,0.840629,0.875421,0.804714,0.028877
1257,"[Fare, Log_Fare]",GB,0.837262,0.848485,0.821549,0.011446
1457,"[Family, Salutation]",GB,0.837262,0.855219,0.811448,0.018713
1057,"[Parch, Salutation_type]",GB,0.836139,0.858586,0.804714,0.022891
21,[Name],SVC,0.835017,0.841751,0.821549,0.009523
71,[Ticket],SVC,0.835017,0.841751,0.821549,0.009523
91,[Cabin],SVC,0.835017,0.841751,0.821549,0.009523
101,[Embarked],SVC,0.835017,0.841751,0.821549,0.009523
151,[Fare_Category],SVC,0.835017,0.841751,0.821549,0.009523
161,[isCabin],SVC,0.835017,0.841751,0.821549,0.009523


### Разделение выборки

In [23]:
#Подготовка с отбором признаков
exclude_col=['Age',]
#exclude_col = ['Name', 'Embarked', 'Cabin', 'isCabin','Salutation', 'Sex', 'Family']
#['Name', 'Embarked', 'Cabin', 'isCabin'] ['Salutation', 'Sex', 'Family']
#prepare_data(raw_train_data,[])

#current_train_data=original_train_data.copy()
#prepare_data(current_train_data,exclude_col)
#train_labels = raw_train_data['Survived']
#current_train_data = current_train_data.drop(['PassengerId', 'Survived'], axis = 1)
#print(current_train_data.columns)
#current_categorical_data_indices = np.array([(column in categorical_data_columns) for column in current_train_data.columns], dtype = bool)
#current_numeric_data_indices = np.array([(column in numeric_data_columns) for column in current_train_data.columns], dtype = bool)
#clf=create_estimator(current_numeric_data_indices,current_categorical_data_indices,v)
#clf_scoring = cross_val_score(clf, current_train_data.values, train_labels, scoring = 'accuracy', cv = 3)

current_train_data=train_data.copy()
current_train_data.drop(exclude_col, axis = 1,inplace=True)
current_categorical_data_indices = np.array([(column in categorical_data_columns) for column in current_train_data.columns], dtype = bool)
current_numeric_data_indices = np.array([(column in numeric_data_columns) for column in current_train_data.columns], dtype = bool)



In [24]:
X_train, X_test, y_train, y_test = train_test_split(current_train_data.values, train_labels, test_size=0.33,shuffle=True, random_state=42)
#stratify=train_labels
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((596, 17), (295, 17), (596,), (295,))

### Test

In [25]:
test_data=prepare_data(original_test_data,'new_features1')
#current_train_data=original_train_data.copy()
#prepare_data(current_train_data,'',exclude_col)
#train_labels = train_data['Survived']
test_data.drop(['PassengerId']+exclude_col, axis = 1,inplace=True)
all_column_test=test_data.columns
#Описание колонок и индексов для FeatureUnion
#Колонки
#Проверка пропусков
NAs = pd.concat([test_data.isnull().sum()], axis=1, keys=['data'])
print('Проверка пропусков',NAs[NAs.sum(axis=1) > 0])

--Original mode--
Проверка пропусков Empty DataFrame
Columns: [data]
Index: []


### оценка алгоритмов

In [26]:
# кросс валидация тратегии
cv_strategy_SKF = model_selection.StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
cv_strategy_KF =model_selection.KFold(n_splits=5, shuffle = True, random_state = 0)
cv_strategy_ShS =model_selection.ShuffleSplit(n_splits=5, train_size=0.5, test_size=.25,random_state=0)
cv_strategy_ShS =model_selection.StratifiedShuffleSplit(n_splits=5, train_size=0.5, test_size=.25,random_state=0)
cv_dict = {'SKF':cv_strategy_SKF,'KF':cv_strategy_KF,'ShS':cv_strategy_ShS,'ShS':cv_strategy_ShS}

In [27]:
classifier_dict = {'RiC':linear_model.RidgeClassifier(),'SVC':svm.SVC(gamma='auto'),'SGD':linear_model.SGDClassifier(),'LR':linear_model.LogisticRegression(solver='lbfgs'),
                   'KNC':neighbors.KNeighborsClassifier(),'DeTree':tree.DecisionTreeClassifier(),
                   'RF':ensemble.RandomForestClassifier(n_estimators=10),'GB':ensemble.GradientBoostingClassifier(),
                   'BG':ensemble.BaggingClassifier(),'NB':naive_bayes.GaussianNB()}

In [28]:
%%time
# кросс валидация оценка

clf_res_list=[]
cv_res_list=[]
mean_res_list=[]
max_res_list=[]
min_res_list=[]
std_res_list=[]
test_acc=[]
for k, v in classifier_dict.items():
    #print("Code : {0}, Value : {1}".format(k, v))
    print('***Classifier*****',k)
    clf=create_estimator(current_numeric_data_indices,current_categorical_data_indices,v)
    for name,val in cv_dict.items():
        clf_scoring = cross_val_score(clf, X_train, y_train, scoring = 'accuracy', cv = val)
        print('--cv--',name,'----',val)
        #print ('mean:{}, max:{}, min:{}, std:{}'.format(clf_scoring.mean(), clf_scoring.max(), clf_scoring.min(), clf_scoring.std()))
        clf_res_list.append(k)
        cv_res_list.append(name)
        mean_res_list.append(clf_scoring.mean())
        max_res_list.append(clf_scoring.max())
        min_res_list.append(clf_scoring.min())
        std_res_list.append(clf_scoring.std())
        clf.fit(X_train, y_train)
        y_pred=clf.predict(X_test)
        test_acc.append(accuracy_score(y_test, y_pred))

***Classifier***** RiC
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=0.5)
***Classifier***** SVC
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=0.5)
***Classifier***** SGD
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=0.5)
***Classifier***** LR
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedSh

In [29]:
res_table=pd.DataFrame({'clf':clf_res_list, 'cv':cv_res_list,'mean':mean_res_list,'max':max_res_list,'min':min_res_list,'std':std_res_list,'test_acc':test_acc})
res_table.sort_values(by=['test_acc'],ascending=False)

,clf,cv,mean,max,min,std,test_acc
3,SVC,SKF,0.830546,0.865546,0.798319,0.026718,0.850847
4,SVC,KF,0.827213,0.882353,0.789916,0.031957,0.850847
5,SVC,ShS,0.829530,0.838926,0.818792,0.008054,0.850847
0,RiC,SKF,0.818782,0.865546,0.789916,0.026423,0.823729
1,RiC,KF,0.828880,0.882353,0.806723,0.028283,0.823729
2,RiC,ShS,0.813423,0.832215,0.805369,0.010738,0.823729
22,GB,KF,0.825504,0.857143,0.798319,0.024573,0.823729
9,LR,SKF,0.827171,0.882353,0.789916,0.032638,0.823729
10,LR,KF,0.835588,0.873950,0.806723,0.023389,0.823729
11,LR,ShS,0.812081,0.838926,0.798658,0.014078,0.823729


### Подбор пареметров для RidgeClassifier

In [30]:
#SGD
#RidgeClassifier

parameters_grid = {
    'model_fitting__alpha' : [1e-3, 1e-2, 1e-1, 1,1e1,1e2], #RC
    #'model_fitting__kernel' : ['linear', 'poly', 'rbf', 'sigmoid'], #GB
    #'model_fitting__class_weight' : [None,'balanced'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 0.5, 12),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 12),
    #'model_fitting__max_features':["log2","sqrt"],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [31]:
clf=create_estimator(current_numeric_data_indices,current_categorical_data_indices,linear_model.RidgeClassifier(fit_intercept=False))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=40)

In [ ]:
%%time
#grid_cv.fit(train_data.values, train_labels)
grid_cv.fit(X_train, y_train)

print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
type(grid_cv.best_estimator_[1])

In [ ]:
#Коэфициенты модели
#grid_cv.best_estimator_.steps[0][1].transformer_list[1][1].steps[0][1].fit_transform(X_train)[:2]
#one_hot_cat_col=grid_cv.best_estimator_.steps[0][1].transformer_list[1][1][1].get_feature_names(categorical_data_columns)
#grid_cv.best_estimator_.steps[0][1].transformer_list[1][1].steps[1][1].categories_
all_column=numeric_data_columns+list(grid_cv.best_estimator_.steps[0][1].transformer_list[1][1].steps[1][1].get_feature_names())
coef=grid_cv.best_estimator_.steps[1][1].coef_[0]
sorted(zip(np.abs(coef),all_column),reverse=True)

### Подбор пареметров для SVC

In [ ]:
#SVC

parameters_grid = {
    'model_fitting__C' : [0.01,0.1, 1, 10, 100,], 
    #'model_fitting__C' : [0.8, 0.85, 0.9],
    'model_fitting__kernel' : ['linear','rbf', 'sigmoid'], 
    'model_fitting__class_weight' : [None,'balanced'],
    'model_fitting__decision_function_shape':['ovo', 'ovr'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 0.5, 12),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 12),
    #'model_fitting__max_features':["log2","sqrt"],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [ ]:
clf=create_estimator(current_numeric_data_indices,current_categorical_data_indices,svm.SVC(gamma='auto'))
cv = model_selection.KFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
#clf.get_params().keys()

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv,iid=True)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=40,iid=True)

In [ ]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

### Подбор пареметров для GB

In [ ]:
#GradientBoosting

parameters_grid = {
    'model_fitting__n_estimators' : [3,4, 8, 16, 32, 64, 100], #GB
    'model_fitting__max_depth' : [3,5,7], #GB
    #'model_fitting__loss' : ["deviance",'exponential'],
    'model_fitting__learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 1, 5),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 5),
    'model_fitting__max_features':['auto',"log2","sqrt",None],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [ ]:
clf=create_estimator(current_numeric_data_indices,current_categorical_data_indices,ensemble.GradientBoostingClassifier())
cv = model_selection.KFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
#clf.get_params().keys()

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv,iid=False)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=100)

In [ ]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
#y_pred=grid_cv.predict(X_test)

In [ ]:
#accuracy_score(y_test, y_pred)

### Подбор пареметров для LR

In [ ]:
#LR

parameters_grid = {
    'model_fitting__C' : [0.001,0.01,0.1,1,10,100], #GB
    #'model_fitting__penalty' : ['l2','l1','none'],
    'model_fitting__solver' : ['newton-cg', 'lbfgs', 'liblinear']
    
}

In [ ]:
clf=create_estimator(current_numeric_data_indices,current_categorical_data_indices,linear_model.LogisticRegression(max_iter=1000))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=100)

In [ ]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

### Подбор пареметров для RF

In [ ]:
#RandomForestClassifier(n_estimators=10)

parameters_grid = {
    'model_fitting__n_estimators' : [1, 2, 4, 8, 16, 32, 64, 100, 200,500], #GB
    #'model_fitting__max_depth' : np.linspace(1, 32, 32, endpoint=True), #GB
    #'model_fitting__loss' : ["deviance",'exponential'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    'model_fitting__min_samples_split': [2,5,7,10],
    'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True),
    #'model_fitting__max_features':["log2","sqrt",'auto'],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
    #'model_fitting__class_weight' : ['balanced',None]
}

In [ ]:
clf=create_estimator(current_numeric_data_indices,current_categorical_data_indices,ensemble.RandomForestClassifier(class_weight='balanced'))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=100)

In [ ]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

## Обучение финального классификатора ALL_TRAIN

In [ ]:
#SVC

parameters_grid = {
    'model_fitting__C' : [0.01,0.1, 1, 10, 100,], 
    #'model_fitting__C' : [0.8, 0.85, 0.9],
    'model_fitting__kernel' : ['linear','rbf', 'sigmoid'], 
    'model_fitting__class_weight' : [None,'balanced'],
    'model_fitting__decision_function_shape':['ovo', 'ovr'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 0.5, 12),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 12),
    #'model_fitting__max_features':["log2","sqrt"],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [ ]:
clf=create_estimator(current_numeric_data_indices,current_categorical_data_indices,svm.SVC(gamma='auto'))
cv = model_selection.KFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv,iid=False)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,iid=False,n_iter=40)

In [ ]:
%%time
grid_cv.fit(current_train_data.values, train_labels)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

### формирование файла Kaggle

In [ ]:
result = grid_cv.predict(current_test_data.values)
submission = pd.DataFrame({'PassengerId':raw_test_data.PassengerId,'Survived':result})
submission.Survived = submission.Survived.astype(int)
print(submission.shape)


In [ ]:
filename = 'titanic17_sf_SVC.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

In [ ]:
#Преобразование категориальных признаков в отдельном df
#dummy_data=pd.get_dummies(data=raw_test_data, columns=categorical_data_columns)
#dummy_index=dummy_data.columns.str.contains('|'.join(categorical_data_columns))
#dummy_data=dummy_data.loc[:,dummy_index]
#Числовые данные в отдельном df
#numeric_data=raw_test_data[numeric_data_columns]
#создаем стандартный scaler
#scaler = StandardScaler()
#scaled_test_data=scaler.transform(numeric_data)
#scaled_test_data = scaler.transform(test_data)
#all_test_data=np.hstack((scaled_test_data,dummy_data.values))

### Стекинг для валидации

In [ ]:
X_train_st, X_test_st, y_train_st, y_test_st = train_test_split(train_data.values, train_labels.values, test_size=0.25, random_state=42)

In [ ]:
X_train_st.shape,y_train_st.shape,y_train_st.shape

In [ ]:
#linear_model.RidgeClassifier(alpha=10.0)
#svm.SVC(gamma='auto',C=0.8, decision_function_shape='ovo', kernel= 'linear')
#
clfs = [create_estimator(numeric_data_indices,categorical_data_indices,svm.SVC(gamma='auto',C=0.8, decision_function_shape='ovo', kernel= 'linear')),
        create_estimator(numeric_data_indices,categorical_data_indices,linear_model.LogisticRegression(solver='liblinear',class_weight=None,tol=1e-05, penalty='l2')),
        create_estimator(numeric_data_indices,categorical_data_indices,linear_model.RidgeClassifier(alpha=10.0)),
        create_estimator(numeric_data_indices,categorical_data_indices,ensemble.GradientBoostingClassifier(learning_rate= 0.1, max_depth= 3, max_features='auto', n_estimators= 32))
       ]

In [ ]:
dataset_blend_train = np.zeros((X_train_st.shape[0], len(clfs)))

In [ ]:
kf = model_selection.StratifiedKFold(n_splits=5,shuffle=False)
for j, clf in enumerate(clfs):
    print (j, clf[1])
    for i, (train_, test_) in enumerate(kf.split(X_train_st, y_train_st)):
        print ("Fold", i)
        X_tr = X_train_st[train_]
        y_tr = y_train_st[train_]
        X_te = X_train_st[test_]
        y_te = y_train_st[test_]
        clf.fit(X_tr, y_tr)
        print('------------------------',type(clf[1]))
        if isinstance(clf[1],svm.SVC) or isinstance(clf[1],linear_model.ridge.RidgeClassifier):
            print('----------SVM/RidgeCl-------')
            y_submission = clf.predict(X_te)
        else:
            print('----------Other-------')
            y_submission = clf.predict_proba(X_te)[:, 1]
        dataset_blend_train[test_, j] = y_submission

In [ ]:
isinstance(clf[1],svm.SVC),
type(clf[1])

In [ ]:
dataset_blend_train.shape

In [ ]:
clf2 = linear_model.RidgeClassifier(class_weight=None,tol=1e-05, random_state=11)
ls = cross_val_score(clf2, dataset_blend_train, y_train_st, cv=5, scoring='accuracy')
print(ls)
np.mean(ls)

In [ ]:
clf2.fit(dataset_blend_train, y_train_st)

In [ ]:
#clf2.predict()

### Тест для стекинга

In [ ]:
#kf = model_selection.StratifiedKFold(n_splits=5,shuffle=False)
X_meta = np.zeros((X_test_st.shape[0],len(clfs)))
for j, clf in enumerate(clfs):
    print (j, clf[1])
    #for i, (train_, test_) in enumerate(kf.split(train_data, train_labels)):
    #    print ("Fold", i)
    #    X_tr = train_data.values[train_]
    #    y_tr = train_labels[train_]
    #    X_te = train_data.values[test_]
    #    y_te = train_labels[test_]
    #    clf_pipeline=create_estimator(numeric_data_indices,categorical_data_indices,clf)
    #    clf_pipeline.fit(X_tr, y_tr)
    print('------------------------',type(clf[1]))
    if isinstance(clf[1],svm.SVC) or isinstance(clf[1],linear_model.ridge.RidgeClassifier):
        print('----------SVM/RidgeCl-------')
        #y_submission = clf_.predict(X_te)
        X_meta[:, j] = clf.predict(X_test_st)
    else:
        print('----------Other-------')
        #y_submission = clf_pipeline.predict_proba(X_te)[:, 1]
        X_meta[:, j] = clf.predict_proba(X_test_st)[:, 1]
    #dataset_blend_train[test_, j] = y_submission

In [ ]:
X_test.shape,X_meta.shape

In [ ]:
pred_meta=clf2.predict(X_meta)

In [ ]:
accuracy_score(y_test_st, pred_meta)

## Стекинг для ответа kaggle

In [ ]:
#linear_model.RidgeClassifier(alpha=10.0)
#svm.SVC(gamma='auto',C=0.8, decision_function_shape='ovo', kernel= 'linear')
#
clfs = [create_estimator(numeric_data_indices,categorical_data_indices,svm.SVC(gamma='auto',C=0.8, decision_function_shape='ovo', kernel= 'linear')),
        create_estimator(numeric_data_indices,categorical_data_indices,linear_model.LogisticRegression(solver='liblinear',class_weight=None,tol=1e-05, penalty='l2')),
        create_estimator(numeric_data_indices,categorical_data_indices,linear_model.RidgeClassifier(alpha=10.0)),
        create_estimator(numeric_data_indices,categorical_data_indices,ensemble.GradientBoostingClassifier(learning_rate= 0.1, max_depth= 3, max_features='auto', n_estimators= 32))
       ]

In [ ]:
X_train_st_kaggle=train_data.values
y_train_st_kaggle=train_labels.values

In [ ]:
dataset_blend_train = np.zeros((X_train_st_kaggle.shape[0], len(clfs)))

In [ ]:
kf = model_selection.StratifiedKFold(n_splits=5,shuffle=False)
for j, clf in enumerate(clfs):
    print (j, clf[1])
    for i, (train_, test_) in enumerate(kf.split(X_train_st_kaggle, y_train_st_kaggle)):
        print ("Fold", i)
        X_tr = X_train_st_kaggle[train_]
        y_tr = y_train_st_kaggle[train_]
        X_te = X_train_st_kaggle[test_]
        y_te = y_train_st_kaggle[test_]
        clf.fit(X_tr, y_tr)
        print('------------------------',type(clf[1]))
        if isinstance(clf[1],svm.SVC) or isinstance(clf[1],linear_model.ridge.RidgeClassifier):
            print('----------SVM/RidgeCl-------')
            y_submission = clf.predict(X_te)
        else:
            print('----------Other-------')
            y_submission = clf.predict_proba(X_te)[:, 1]
        dataset_blend_train[test_, j] = y_submission

In [ ]:
clf_meta_kaggle = linear_model.RidgeClassifier(class_weight=None,tol=1e-05, random_state=11)
clf_meta_kaggle.fit(dataset_blend_train, y_train_st_kaggle)

### Стекинг test для kaggle

In [ ]:
X_test_st_kaggle=test_data.values

In [ ]:
#kf = model_selection.StratifiedKFold(n_splits=5,shuffle=False)
X_meta = np.zeros((X_test_st_kaggle.shape[0],len(clfs)))
for j, clf in enumerate(clfs):
    print (j, clf[1])
    #for i, (train_, test_) in enumerate(kf.split(train_data, train_labels)):
    #    print ("Fold", i)
    #    X_tr = train_data.values[train_]
    #    y_tr = train_labels[train_]
    #    X_te = train_data.values[test_]
    #    y_te = train_labels[test_]
    #    clf_pipeline=create_estimator(numeric_data_indices,categorical_data_indices,clf)
    #    clf_pipeline.fit(X_tr, y_tr)
    print('------------------------',type(clf[1]))
    if isinstance(clf[1],svm.SVC) or isinstance(clf[1],linear_model.ridge.RidgeClassifier):
        print('----------SVM/RidgeCl-------')
        #y_submission = clf_.predict(X_te)
        X_meta[:, j] = clf.predict(X_test_st_kaggle)
    else:
        print('----------Other-------')
        #y_submission = clf_pipeline.predict_proba(X_te)[:, 1]
        X_meta[:, j] = clf.predict_proba(X_test_st_kaggle)[:, 1]
    #dataset_blend_train[test_, j] = y_submission

In [ ]:
X_meta.shape

In [ ]:
result=clf_meta_kaggle.predict(X_meta)
submission = pd.DataFrame({'PassengerId':raw_test_data.PassengerId,'Survived':result})
submission.Survived = submission.Survived.astype(int)
print(submission.shape)

In [ ]:
filename = 'titanic16_stack.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

In [ ]:
string_data_columns = ['Name','Ticket','Cabin']
categorical_data_columns = ['Pclass','Sex','Embarked','Salutation','Age_Range', 'Fare_Category','Family_Size','Salutation_type']
numeric_data_columns = ['Age', 'SibSp', 'Parch', 'Fare','Family','isCabin','Log_Fare']
#numeric_data_columns = ['Family']